In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

/var/folders/3w/1t3nmp6d3d75s59yvygy1pyh0000gn/T/ipykernel_9601/2027532251.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),

                                dcc.Dropdown(id = 'site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                             value = 'ALL',
                                             placeholder = 'Select a Site'),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                ),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id ='success-pie-chart', component_property ='figure'),
              Input(component_id ='site-dropdown', component_property ='value'))

def pie_chart(site_dropdown):
    df = spacex_df
    if site_dropdown == 'ALL':
        figure = px.pie(df, names ='Launch Site', 
                         values ='class',
                         title ='Successful Launches per Site')
        return figure
    else:
        site = df[df['Launch Site'] == site_dropdown]
        figure = px.pie(site, names ='class', 
                         title='Successful vs Failed Launches')
        return figure


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id ='success-payload-scatter-chart',component_property ='figure'),  
              [Input(component_id ='site-dropdown',component_property ='value'),
               Input(component_id ='payload-slider',component_property ='value')])

def scatter_plot(site_dropdown, payload_slider):
    df = spacex_df
    if site_dropdown == 'ALL':
        figure = px.scatter(df, 
                              x = 'Payload Mass (kg)',
                              y = 'class',
                              color ='Booster Version Category',
                              title ='Payload and Success Rate')
        return figure
    else:
        site = df[df['Launch Site'] == site_dropdown]
        figure = px.scatter(site, 
                               x ='Payload Mass (kg)',
                               y ='class',
                               color ='Booster Version Category',
                               title ='Payload and Success Rate per Site')
        return figure

# Run the app
if __name__ == '__main__':
    app.run_server()